# 모듈 불러오기

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
%matplotlib inline
from scipy import fftpack, fft
import numpy as np
from PIL import Image
import math

In [0]:
PI = math.pi

# 데이터 불러오기

In [0]:
img_data_countour = list()
img_data = list()
for num in range(0,10):
  
  tmp_img_countour = Image.open(f'/content/drive/My Drive/_camp/mnist/{num}_contour.png의 사본')
  tmp_img = Image.open(f'/content/drive/My Drive/_camp/mnist/{num}.png의 사본')
  
  tmp_img_countour = 255 - np.array(tmp_img_countour.convert('L'))
  tmp_img = 255 - np.array(tmp_img.convert('L'))
  
  img_data_countour.append(tmp_img_countour)
  img_data.append(tmp_img)

FileNotFoundError: ignored

## 푸리에 시리즈 구현해보기

In [0]:
# 벡터 그리기
def dimensionExpansion(img_num):
  height = img_data_countour[img_num].shape[0]
  width = img_data_countour[img_num].shape[1]

  all_vectors = list()
  for y in range(img_data_countour[img_num].shape[0]):
    for x in range(img_data_countour[img_num].shape[1]):
      if img_data_countour[img_num][y,x]:        
        # 이미지 상에서 (x,y)로 나타내자
        all_vectors.append(    [ x-calculateCenter(img_num)[0], (height - y) - calculateCenter(img_num)[1] ])
  return np.array(all_vectors)

<hr>

# 해봅시다

In [0]:
all_vectors = dimensionExpansion(0) 

IndexError: ignored

In [0]:
# 수치형으로 변환
ori_data = all_vectors
ori_data = ori_data.astype(int) 

In [0]:
# 기준잡기, y=0, x=max
y_0 = ori_data[ori_data[:,1]==0]
# y_0_idx = y_0.argmax()
y_0_val = y_0.max(0)

# 정리
base = ori_data[ori_data[:,1]==0].max(0)
base

In [0]:
# 기준잡기 clean ver
def find_base(ori_data):
  ori_data = ori_data.astype(int)
  base = ori_data[ori_data[:,1]==0].max(0)
  return np.array([base[0]]), base[1]
find_base(ori_data)

In [0]:
# data 복사본
tmp = ori_data
tmp1 =np.array([6,0])
tmp2 =np.array([7,0])
tmp3 =np.array([8,0])
tmp4 =np.array([9,0])
tmp5 =np.array([10,0])
tmp6 =np.array([15,0])
tmp = np.array([tmp1, tmp2, tmp3, tmp4, tmp5, tmp6])

# 한 점에서 x축으로 이어진 점들을 본다
x_left = True
x_left_idx = base[0]-1
x_right = True
x_right_idx = base[0]

# x_list = list()
x_list = np.array( [base] )

while x_left:
  # print(x_left_idx)

  n = np.where( (tmp[:,1] == base[1]) & (tmp[:,0] == x_left_idx) )
  print(n[0])
  if len(n[0]) == 1:
    # print("in")
    x_left_idx -= 1
    x_list = np.append(x_list, tmp[n[0]], axis=0)
    tmp = np.delete(tmp, [n[0]], 0)
  else:
    x_left = False
while x_right:
  # print(x_right_idx)
  n = np.where( (tmp[:,1] == base[1]) & (tmp[:,0] == x_right_idx) )
  # print(n)
  if len(n[0]) == 1:
    x_right_idx += 1
    x_list = np.append(x_list, tmp[n[0]], axis=0)
    tmp = np.delete(tmp, [n[0]], 0)
  else:
    x_right = False

print(x_list)
print(tmp)

NameError: ignored

# 로직 정리
- input:
  - ori_data : 전체 데이터셋(중간의 경우, 나왔던 것은 제거)
  - base_x_list : 
    - 처음일 경우, y가 0이면서 x가 최대인 시작base점
    - 중간일 경우, 이전 라인에서 읽은 x점들
  - y : 읽는 y라인

- 1. base점 선정
  - 1.1. base_x_list 한 개일 경우
    - base를 해당 x좌표와 y로 생성
  - 1.2. base_x_list 여러 개일 경우
    - 현재 y라인의 x좌표를 검사하면서 같은 것이 있는지 확인
    - add.1) 양쪽 끝 의 대각선도 검사해야 함( 중간에서 겹치는 것이 없다면)

- 2. 좌측, 우측 search
  - 2.1. 먼저 base 점부터 result_list 삽입
  - 2.2. 좌측, 우측 순으로 찾으며 result_list 삽입
  - 2.3. 매번 해당 인덱스를 ori_data에서 지워야 한다.

- 3. result_list 반환
  - 3.1. 반환하기 전에 정렬을 해주어야 한다.
  - 3.2. x좌표만 따서 따로 반환해야 기능적으로 반복하기 좋다.

## add
- 1) 양쪽 끝 의 대각선도 검사해야 함( 중간에서 겹치는 것이 없다면)
  - 1단계에서 조건절 추가, 아무것도 찾지 못했을 때
    - 좌측보다 우측이 우선 조건
    - try_except로...


## 고민
- 리턴이 자기를 불러야되나? 
  - 재귀로 해서 제일 큰 조건이 ori_data에 값이 있냐 없냐로..?

In [0]:
# 아치형 => 가능
tmp = np.array([np.array([6,0]), 
                np.array([7,0]),
                np.array([8,0]),
                np.array([9,0]),
                np.array([10,0]),

                np.array([8,1]),
                np.array([9,1]),
                np.array([10,1]),

                np.array([11,2]),
                np.array([12,2]),

                np.array([12,3]),

                np.array([12,4]),
                np.array([13,4]),

                np.array([14,3]),

                np.array([14,2]),
                np.array([15,2]),

                np.array([15,1]),
                np.array([16,1]),

                np.array([16,0]),
                np.array([17,0]),
                
                ] )

In [0]:
# 원형 => 성공
tmp = np.array([np.array([6,0]), 
                np.array([7,0]),
                np.array([8,0]),
                np.array([9,0]),
                np.array([10,0]),

                np.array([8,1]),
                np.array([9,1]),
                np.array([10,1]),

                np.array([11,2]),
                np.array([12,2]),

                np.array([12,3]),

                np.array([12,4]),
                np.array([13,4]),

                np.array([14,3]),

                np.array([14,2]),
                np.array([15,2]),

                np.array([15,1]),
                np.array([16,1]),

                np.array([16,0]),
                np.array([17,0]),

                np.array([18,-1]),

                np.array([18,-2]),

                np.array([18,-3]),
                np.array([17,-3]),

                np.array([16,-4]),
                np.array([17,-4]),

                np.array([15,-5]),
                np.array([16,-5]),

                np.array([14,-6]),

                np.array([13,-7]),
                np.array([12,-7]),
                np.array([11,-7]),

                np.array([10,-6]),

                np.array([10,-5]),
                
                np.array([10,-4]),
                np.array([9,-4]),

                np.array([9,-3]),
                np.array([8,-3]),

                np.array([7,-2]),
                np.array([8,-2]),
                np.array([9,-2]),

                np.array([8,-1]),
                np.array([7,-1]),
                
                ] )

In [0]:
# clean ver -> not clean...
# tmp = ori_data

# print(tmp)

def find_close_x(ori_data, base_x_list, y, y_direction, res, cnt=0, idx = 0):
  if len(ori_data) > 0 and cnt < 3:
    if len(base_x_list) > 1:
      for x in base_x_list:
        n = np.where( (ori_data[:,1] == y) & (ori_data[:,0] == x) )
        if len(n[0]) == 1:
          base_x = ori_data[n[0]][0][0]
          break
    
    else: # base_x_list => 1개
      n = np.where( (ori_data[:,1] == y) & (ori_data[:,0] == base_x_list[0]) )
      if len(n[0]) == 1:
        base_x = ori_data[n[0]][0][0]
      
    try:
      base = np.array([base_x, y])
    except:
      n = np.where( (ori_data[:,1] == y) & (ori_data[:,0] == base_x_list[-1]+1) )
      if len(n[0]) == 1:
        base_x = ori_data[n[0]][0][0]
        # print("우측을 보자~~")
        # print(base_x)
      else:
        n = np.where( (ori_data[:,1] == y) & (ori_data[:,0] == base_x_list[0]-1) )
        if len(n[0]) == 1:
          base_x = ori_data[n[0]][0][0]
          # print("좌측을 보자~~")
          # print(base_x)
        else:
          # return "need U-turn"
          if cnt == 0:
            y_direction *= -1
          y += y_direction
          cnt += 1

          # print("유턴 구간")
          # print(y_direction)
          # print(y)
          return find_close_x(ori_data, base_x_list, y, y_direction, cnt=cnt, idx=idx, res=res)
      base = np.array([base_x, y])
    # print("베이스 지점~~")
    # print(base)
        
    x_left = True
    x_right = True
    x_left_idx = base[0]-1
    x_right_idx = base[0]+1

    result_list = np.array([base])
    n = np.where( (ori_data[:,1] == y) & (ori_data[:,0] == base[0]) )
    ori_data = np.delete(ori_data, n[0], 0)

    while x_left:
      n = np.where( (ori_data[:,1] == y) & (ori_data[:,0] == x_left_idx) )
      if len(n[0]) == 1:
        x_left_idx -= 1
        result_list = np.append(result_list, ori_data[n[0]], axis=0)
        ori_data = np.delete(ori_data, n[0], 0)
      else:
        x_left = False
      
    while x_right:
      n = np.where( (ori_data[:,1] == y) & (ori_data[:,0] == x_right_idx) )
      if len(n[0]) == 1:
        x_right_idx += 1
        result_list = np.append(result_list, ori_data[n[0]], axis=0)
        ori_data = np.delete(ori_data, n[0], 0)
      else:
        x_right = False
    
    result_list = result_list[np.argsort(result_list[:, 0])]
    x_list = result_list[:,0]
    
    # print("찾은 y 줄")
    # print(y)
    # print("찾은 x 리스트")
    # print(x_list)
    # print("찾아야할 데이터들")
    # print(ori_data)
    # print("="*50)
    

    for x in x_list:
        # print(type(x)) # <class 'numpy.int32'>
        # print(np.array([x, y, idx]))
        res = np.append( res, np.array([[x, y, idx]]), axis=0 )
        idx += 1

    y += y_direction

    # idx += len(x_list)


    return find_close_x(ori_data, x_list, y, y_direction, cnt=0, idx=idx, res=res)

  else: # ori_data가 비어있을 때
    if len(ori_data) == 0:
      print("총 검사 포인트 갯수")
      print(idx)
    #   return "empty ori_data!!"
      print("thank you! perfect")
      print(res)
      return res
    else:
      
      return "Error: tracking stopped!!"


arr = np.array([[0,0,0]]) 

find_close_x(ori_data, find_base(ori_data)[0], find_base(ori_data)[1], +1, res=arr, cnt=0, idx=0)
# find_close_x(tmp, np.array([17]), 0, +1, res=arr, cnt=0, idx=0)

NameError: ignored